# Navigation

---

Congratulations for completing the first project of the [Deep Reinforcement Learning Nanodegree](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893)!  In this notebook, you will learn how to control an agent in a more challenging environment, where it can learn directly from raw pixels!  **Note that this exercise is optional!**

### 1. Start the Environment

We begin by importing some necessary packages.  If the code cell below returns an error, please revisit the project instructions to double-check that you have installed [Unity ML-Agents](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Installation.md) and [NumPy](http://www.numpy.org/).

In [1]:
# from unityagents import UnityEnvironment
# import numpy as np
# import matplotlib.pyplot as plt
# %matplotlib inline

Next, we will start the environment!  **_Before running the code cell below_**, change the `file_name` parameter to match the location of the Unity environment that you downloaded.

- **Mac**: `"path/to/VisualBanana.app"`
- **Windows** (x86): `"path/to/VisualBanana_Windows_x86/Banana.exe"`
- **Windows** (x86_64): `"path/to/VisualBanana_Windows_x86_64/Banana.exe"`
- **Linux** (x86): `"path/to/VisualBanana_Linux/Banana.x86"`
- **Linux** (x86_64): `"path/to/VisualBanana_Linux/Banana.x86_64"`
- **Linux** (x86, headless): `"path/to/VisualBanana_Linux_NoVis/Banana.x86"`
- **Linux** (x86_64, headless): `"path/to/VisualBanana_Linux_NoVis/Banana.x86_64"`

For instance, if you are using a Mac, then you downloaded `VisualBanana.app`.  If this file is in the same folder as the notebook, then the line below should appear as follows:
```
env = UnityEnvironment(file_name="VisualBanana.app")
```

In [2]:
# env = UnityEnvironment(file_name="VisualBanana_Linux/Banana.x86_64")

Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [3]:
# # get the default brain
# brain_name = env.brain_names[0]
# brain = env.brains[brain_name]

### 2. Examine the State and Action Spaces

The simulation contains a single agent that navigates a large environment.  At each time step, it has four actions at its disposal:
- `0` - walk forward 
- `1` - walk backward
- `2` - turn left
- `3` - turn right

The environment state is an array of raw pixels with shape `(1, 84, 84, 3)`.  *Note that this code differs from the notebook for the project, where we are grabbing **`visual_observations`** (the raw pixels) instead of **`vector_observations`**.* A reward of `+1` is provided for collecting a yellow banana, and a reward of `-1` is provided for collecting a blue banana. 

Run the code cell below to print some information about the environment.

In [4]:
# # reset the environment
# env_info = env.reset(train_mode=True)[brain_name]

# # number of agents in the environment
# print('Number of agents:', len(env_info.agents))

# # number of actions
# action_size = brain.vector_action_space_size
# print('Number of actions:', action_size)

# # examine the state space 
# state = env_info.visual_observations[0][0]
# print('States look like:')
# plt.imshow(np.squeeze(state))
# plt.show()
# state_size = state.shape
# print('States have shape:', state.shape)

### 3. Take Random Actions in the Environment

In the next code cell, you will learn how to use the Python API to control the agent and receive feedback from the environment.

Once this cell is executed, you will watch the agent's performance, if it selects an action (uniformly) at random with each time step.  A window should pop up that allows you to observe the agent, as it moves through the environment.  

Of course, you'll have to change the code so that the agent is able to use its experience to gradually choose better actions when interacting with the environment!

In [5]:
# env_info = env.reset(train_mode=False)[brain_name] # reset the environment
# state = env_info.visual_observations[0]            # get the current state
# score = 0                                          # initialize the score
# while True:
#     action = np.random.randint(action_size)        # select an action
#     env_info = env.step(action)[brain_name]        # send the action to the environment
#     next_state = env_info.visual_observations[0]   # get the next state
#     reward = env_info.rewards[0]                   # get the reward
#     done = env_info.local_done[0]                  # see if episode has finished
#     score += reward                                # update the score
#     state = next_state                             # roll over the state to next time step
#     if done:                                       # exit loop if episode finished
#         break
    
# print("Score: {}".format(score))

When finished, you can close the environment.

In [6]:
# env.close()

### 4. It's Your Turn!

Now it's your turn to train your own agent to solve the environment!  When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]
```

In [7]:
import os
from collections import deque
from typing import List

import numpy as np
import pickle
import torch
from unityagents import UnityEnvironment
from skimage import color

from agent import Agent
from model import QNetwork, QNetworkWithPixels, QNetworkWithoutPixels
from scaler import Scaler

In [8]:
%load_ext autoreload
%autoreload 2

In [9]:
def encode_state(state: np.ndarray) -> np.ndarray:
    return np.transpose(state, (2, 0, 1))

In [10]:
def train(
    env,
    brain_name: str,
    agent: Agent,
    n_episodes: int=2000,
    max_timestamp_per_episode: int=1000,
    eps_start: float=1.0,
    eps_end: float=0.01,
    eps_decay: float=0.995,
):
    scores = []
    scores_window = deque(maxlen=100)
    eps = eps_start
    
    for episode_idx in range(1, n_episodes + 1):
        env_info = env.reset(train_mode=True)[brain_name]
        state = env_info.visual_observations[0][0]
        encoded_state = encode_state(state)
        score = 0
        for timestamp_idx in range(max_timestamp_per_episode):
            action = agent.act(encoded_state, eps)
            env_info = env.step(action)[brain_name]
            (
                next_state,
                reward,
                done,
            ) = (
                env_info.visual_observations[0][0],
                env_info.rewards[0],
                env_info.local_done[0],
            )
            encoded_next_state = encode_state(next_state)
            agent.step(encoded_state, action, reward, encoded_next_state, done)
            encoded_state = encoded_next_state
            score += reward
            if done:
                break
        
        scores_window.append(score)
        scores.append(score)
        eps = max(eps_end, eps * eps_decay)
        average_score = np.mean(scores_window)
        
        if episode_idx % 100 == 0:
            print(f"Episode {episode_idx} -- Average score: {average_score}")
            
        if average_score >= 20.0:
            print(f"Problem solved within {episode_idx} episodes.")
            print(f"Average score: {average_score}")
            break
    return 

In [11]:
def inference(
    env,
    brain_name: str,
    agent: Agent,
    eps: float=0.01,  # use the eps_end in training
) -> None:
    env_info = env.reset(train_mode=False)[brain_name]
    state = env_info.vector_observations[0]
    encoded_state = encode_state(state)
    score = 0
    while True:
        action = agent.act(encoded_state, eps)
        env_info = env.step(action)[brain_name]
        (
            next_state,
            reward,
            done,
        ) = (
            env_info.vector_observations[0],
            env_info.rewards[0],
            env_info.local_done[0],
        )
        encoded_next_state = encode_state(next_state)
        encoded_state = encoded_next_state
        score += reward
        if done:
            break

    print("Score: {}".format(score))

In [12]:
env = UnityEnvironment(file_name="VisualBanana_Linux/Banana.x86_64")
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		
Unity brain name: BananaBrain
        Number of Visual Observations (per agent): 1
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 0
        Number of stacked Vector Observation: 1
        Vector Action space type: discrete
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


In [13]:
# state_size = (84, 84)

# OVERWRITE = True
# MODEL_FILE_PATH = "qnetwork_with_pixels_local.pt"

# agent = Agent(
#     state_size=state_size,
#     action_size=brain.vector_action_space_size,
#     seed=514,
# )

# if not OVERWRITE and os.path.exists(MODEL_FILE_PATH):
#     agent.qnetwork_local.load_state_dict(torch.load(MODEL_FILE_PATH))
# else:
#     agent = Agent(
#         state_size=state_size,
#         action_size=brain.vector_action_space_size,
#         seed=514,
#         model_cls=QNetworkWithPixels,
#     )
#     train(env, brain_name, agent)
#     torch.save(agent.qnetwork_local.state_dict(), MODEL_FILE_PATH)

In [14]:
state_size = (84, 84)

In [15]:
agent = Agent(
    state_size=state_size,
    action_size=brain.vector_action_space_size,
    seed=514,
    model_cls=QNetworkWithPixels,
)

In [16]:
train(env, brain_name, agent)

Episode 100 -- Average score: -0.07
Episode 200 -- Average score: 0.07
Episode 300 -- Average score: -0.28
Episode 400 -- Average score: 0.11
Episode 500 -- Average score: 0.0
Episode 600 -- Average score: 0.15
Episode 700 -- Average score: 0.04
Episode 800 -- Average score: 0.06


KeyboardInterrupt: 